In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
import re
from bs4 import BeautifulSoup
 

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\saira\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
! pip install bs4 # in case you don't have it installed

# Dataset: https://web.archive.org/web/20201127142707if_/https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz

ERROR: Invalid requirement: '#'


## Read Data

In [3]:
df = pd.read_csv('D:/Studies/NLP/NLPHW1/dataset/amazon_reviews_us_Office_Products_v1_00.tsv', sep='\t',  on_bad_lines='skip')

C:\Users\saira\AppData\Local\Temp\ipykernel_31388\2628411588.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('D:/Studies/NLP/NLPHW1/dataset/amazon_reviews_us_Office_Products_v1_00.tsv', sep='\t',  on_bad_lines='skip')


## Keep Reviews and Ratings

In [4]:
target_df = df[["star_rating", "review_body"]]

 ## We form two classes and select 50000 reviews randomly from each class.



In [5]:
class_list = []
neg = [1, 2, 3] #class1
pos = [4, 5] #class2
for i in target_df["star_rating"]:
    if (i == 5 or i == 4):
        class_list.append("pos")
    elif (i == 1 or i == 2 or i ==3):
        class_list.append("neg")
    else:
        try:
            if (int(i) == 5 or int(i) == 4):
                class_list.append("pos")
            elif (int(i) == 1 or int(i) == 2 or int(i) ==3):
                class_list.append("neg")
        except:
            class_list.append("XO")
target_df["Classes"] = class_list
rmd = ["XO"]
target_df = target_df[~target_df['Classes'].isin(rmd)]
neg_df = target_df.query('(Classes != "pos")').sample(n=50000)
pos_df = target_df.query('(Classes != "neg")').sample(n=50000)
frames = [neg_df, pos_df]
result = pd.concat(frames)
final_df = result.sample(frac=1)

C:\Users\saira\AppData\Local\Temp\ipykernel_31388\3200697383.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_df["Classes"] = class_list


# Data Cleaning & Pre-Processing



In [6]:
final_df["review_body"] = final_df["review_body"].str.lower()
test = final_df
for i in range(len(test)):
    test.at[test.index[i],'review_body'] = re.sub(r'[^a-zA-Z0-9" "]', "", str(test.at[test.index[i],'review_body']))# removing no alphanumeric
    test.at[test.index[i],'review_body'] = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", str(test.at[test.index[i],'review_body'])) #removing URLS
 

## remove the stop words 

In [7]:
from nltk.tokenize import word_tokenize #stopwords
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
teest = final_df
for i in range(len(teest)):
    words = word_tokenize(str(teest.loc[teest.index[i],'review_body']))
    wordsFiltered = []
    for w in words:
        if w not in stops:
            wordsFiltered.append(w)
    teest.at[teest.index[i],'review_body'] = ' '.join(wordsFiltered)
 

## perform lemmatization  

In [8]:
from nltk.tokenize import word_tokenize #lemmatization
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
teeest = final_df
for i in range(len(teeest)):
    words = word_tokenize(str(teeest.loc[teeest.index[i],'review_body']))
    newwords=[]
    for w in words:
        newwords.append(wnl.lemmatize(w))
    teeest.at[teeest.index[i],'review_body'] = ' '.join(newwords)


In [9]:
final_df['label'] = final_df['Classes'].map({'pos':1, 'neg':0})
X = final_df.review_body
Y = final_df.label

# TF-IDF and BoW Feature Extraction

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

vectorizer = TfidfVectorizer()
data1 = vectorizer.fit_transform(X)
# print(vectorizer.get_feature_names_out())
# print(vectorizer.vocabulary_)
# print()
bow_vectorizer = CountVectorizer()
data2 = bow_vectorizer.fit_transform(X)
# print(bow_vectorizer.get_feature_names_out())
# print(bow_vectorizer.vocabulary_)

# Train Test Split

In [11]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
#train test split for tfidf
xtrain, xtest, ytrain, ytest = train_test_split(data1, Y, test_size=0.2, random_state=2)
#train test split for bow
x1train, x1test, y1train, y1test = train_test_split(data2, Y, test_size=0.2, random_state=2)

# Perceptron Using Both Features

In [12]:
from sklearn.linear_model import Perceptron 
#tfidf Perceptron
tf_pc = Perceptron(random_state=42, max_iter=2500).fit(xtrain, ytrain)
tf_pc_predict = tf_pc.predict(xtest)
print('\n Accuracy of the tfidf classifier is',metrics.accuracy_score(ytest,tf_pc_predict))  #tfidf Perceptron
print('\n Confusion matrix')
print(metrics.confusion_matrix(ytest,tf_pc_predict))
print('\n The value of Precision', metrics.precision_score(ytest,tf_pc_predict))
print('\n The value of Recall', metrics.recall_score(ytest,tf_pc_predict))
print("\nF1 Score: ",metrics.f1_score(ytest, tf_pc_predict))
print("\nClassification Report: \n",metrics.classification_report(ytest, tf_pc_predict))
print("\n*****************************************************\n")
#BOW Perceptron
bow_pc = Perceptron(random_state=42, max_iter=2500).fit(x1train, y1train)
bow_pc_predict = bow_pc.predict(x1test)
print('\n Accuracy of the bow classifier is',metrics.accuracy_score(y1test,bow_pc_predict)) #BOW Perceptron
print('\n Confusion matrix')
print(metrics.confusion_matrix(y1test,bow_pc_predict))
print('\n The value of Precision', metrics.precision_score(y1test,bow_pc_predict))
print('\n The value of Recall', metrics.recall_score(y1test,bow_pc_predict))
print("\nF1 Score: ",metrics.f1_score(y1test, bow_pc_predict))
print("\nClassification Report: \n",metrics.classification_report(y1test, bow_pc_predict))


 Accuracy of the tfidf classifier is 0.7956

 Confusion matrix
[[7949 1965]
 [2123 7963]]

 The value of Precision 0.8020749395648671

 The value of Recall 0.7895102121752925

F1 Score:  0.7957429799140602

Classification Report: 
               precision    recall  f1-score   support

           0       0.79      0.80      0.80      9914
           1       0.80      0.79      0.80     10086

    accuracy                           0.80     20000
   macro avg       0.80      0.80      0.80     20000
weighted avg       0.80      0.80      0.80     20000


*****************************************************


 Accuracy of the bow classifier is 0.796

 Confusion matrix
[[7887 2027]
 [2053 8033]]

 The value of Precision 0.7985089463220676

 The value of Recall 0.7964505254808646

F1 Score:  0.7974784076243423

Classification Report: 
               precision    recall  f1-score   support

           0       0.79      0.80      0.79      9914
           1       0.80      0.80      0.80  

# Logistic Regression Using Both Features

In [13]:
from sklearn.linear_model import LogisticRegression 
#tfidf logstic regression
lr = LogisticRegression(C=100, random_state=1, solver='saga', multi_class='auto', max_iter=5000).fit(xtrain, ytrain)
tf_predict = lr.predict(xtest)
print('\n Accuracy of the tfidf classifier is',metrics.accuracy_score(ytest,tf_predict))  #tfidf logstic regression
print('\n Confusion matrix')
print(metrics.confusion_matrix(ytest,tf_predict))
print('\n The value of Precision', metrics.precision_score(ytest,tf_predict))
print('\n The value of Recall', metrics.recall_score(ytest,tf_predict))
print("\nF1 Score: ",metrics.f1_score(ytest, tf_predict))
print("\nClassification Report: \n",metrics.classification_report(ytest, tf_predict))
print("\n*****************************************************\n")
#bow logstic regression
lr1 = LogisticRegression(C=100, random_state=1, solver='saga', multi_class='auto', max_iter=4000).fit(x1train, y1train)
bow_predict = lr1.predict(x1test)
print('\n Accuracy of the bow classifier is',metrics.accuracy_score(y1test,bow_predict))  #bow logstic regression
print('\n Confusion matrix')
print(metrics.confusion_matrix(y1test,bow_predict))
print('\n The value of Precision', metrics.precision_score(y1test,bow_predict))
print('\n The value of Recall', metrics.recall_score(y1test,bow_predict))
print("\nF1 Score: ",metrics.f1_score(y1test, bow_predict))
print("\nClassification Report: \n",metrics.classification_report(y1test, bow_predict))


 Accuracy of the tfidf classifier is 0.8276

 Confusion matrix
[[8223 1691]
 [1757 8329]]

 The value of Precision 0.8312375249500998

 The value of Recall 0.8257981360301407

F1 Score:  0.8285089028150802

Classification Report: 
               precision    recall  f1-score   support

           0       0.82      0.83      0.83      9914
           1       0.83      0.83      0.83     10086

    accuracy                           0.83     20000
   macro avg       0.83      0.83      0.83     20000
weighted avg       0.83      0.83      0.83     20000


*****************************************************


 Accuracy of the bow classifier is 0.8453

 Confusion matrix
[[8186 1728]
 [1366 8720]]

 The value of Precision 0.8346094946401225

 The value of Recall 0.8645647432084077

F1 Score:  0.8493230739261712

Classification Report: 
               precision    recall  f1-score   support

           0       0.86      0.83      0.84      9914
           1       0.83      0.86      0.85 

# Naive Bayes Using Both Features

In [14]:
from sklearn.naive_bayes import MultinomialNB
#Bayesian TFIDF
NBmodel = MultinomialNB().fit(xtrain, ytrain) 
predicted = NBmodel.predict(xtest)
print('\n Accuracy of the tfidf classifier is',metrics.accuracy_score(ytest,predicted))
print('\n Confusion matrix')
print(metrics.confusion_matrix(ytest,predicted))
print('\n The value of Precision', metrics.precision_score(ytest,predicted))
print('\n The value of Recall', metrics.recall_score(ytest,predicted))
print("\nF1 Score: ",metrics.f1_score(ytest, predicted))
print("\nClassification Report: \n",metrics.classification_report(ytest, predicted))
print("\n*****************************************************\n")
#Bayesian with BOW
NBmodel1 = MultinomialNB().fit(x1train, y1train)
predicted1 = NBmodel1.predict(x1test)
print('\n Accuracy of the bow classifier is',metrics.accuracy_score(y1test,predicted1))
print('\n Confusion matrix')
print(metrics.confusion_matrix(y1test,predicted1))
print('\n The value of Precision', metrics.precision_score(y1test,predicted1))
print('\n The value of Recall', metrics.recall_score(y1test,predicted1))
print("\nF1 Score: ",metrics.f1_score(y1test, predicted1))
print("\nClassification Report: \n",metrics.classification_report(y1test, predicted1))


 Accuracy of the tfidf classifier is 0.81935

 Confusion matrix
[[8477 1437]
 [2176 7910]]

 The value of Precision 0.8462608323526265

 The value of Recall 0.7842554035296451

F1 Score:  0.8140791437245921

Classification Report: 
               precision    recall  f1-score   support

           0       0.80      0.86      0.82      9914
           1       0.85      0.78      0.81     10086

    accuracy                           0.82     20000
   macro avg       0.82      0.82      0.82     20000
weighted avg       0.82      0.82      0.82     20000


*****************************************************


 Accuracy of the bow classifier is 0.80985

 Confusion matrix
[[7622 2292]
 [1511 8575]]

 The value of Precision 0.7890862243489464

 The value of Recall 0.8501883799325798

F1 Score:  0.8184985443611893

Classification Report: 
               precision    recall  f1-score   support

           0       0.83      0.77      0.80      9914
           1       0.79      0.85      0.8

# SVM Using Both Features

In [15]:
from sklearn import svm
#tfidf svm
tf_svm = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto').fit(xtrain, ytrain)
tf_svm_predict = tf_svm.predict(xtest)
print('\n Accuracy of the tfidf classifier is',metrics.accuracy_score(ytest,tf_svm_predict))  #tfidf Perceptron
print('\n Confusion matrix')
print(metrics.confusion_matrix(ytest,tf_svm_predict))
print('\n The value of Precision', metrics.precision_score(ytest,tf_svm_predict))
print('\n The value of Recall', metrics.recall_score(ytest,tf_svm_predict))
print("\nF1 Score: ",metrics.f1_score(ytest, tf_svm_predict))
print("\nClassification Report: \n",metrics.classification_report(ytest, tf_svm_predict))
print("\n*****************************************************\n")
#bow svm
bow_svm = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto').fit(x1train, y1train)
bow_svm_predict = bow_svm.predict(x1test)
print('\n Accuracy of the bow classifier is',metrics.accuracy_score(y1test,bow_svm_predict)) #BOW Perceptron
print('\n Confusion matrix')
print(metrics.confusion_matrix(y1test,bow_svm_predict))
print('\n The value of Precision', metrics.precision_score(y1test,bow_svm_predict))
print('\n The value of Recall', metrics.recall_score(y1test,bow_svm_predict))
print("\nF1 Score: ",metrics.f1_score(y1test, bow_svm_predict))
print("\nClassification Report: \n",metrics.classification_report(y1test, bow_svm_predict))


 Accuracy of the tfidf classifier is 0.84785

 Confusion matrix
[[8479 1435]
 [1608 8478]]

 The value of Precision 0.8552405931604963

 The value of Recall 0.8405710886377157

F1 Score:  0.8478423921196059

Classification Report: 
               precision    recall  f1-score   support

           0       0.84      0.86      0.85      9914
           1       0.86      0.84      0.85     10086

    accuracy                           0.85     20000
   macro avg       0.85      0.85      0.85     20000
weighted avg       0.85      0.85      0.85     20000


*****************************************************


 Accuracy of the bow classifier is 0.83375

 Confusion matrix
[[8024 1890]
 [1435 8651]]

 The value of Precision 0.8207001233279575

 The value of Recall 0.8577235772357723

F1 Score:  0.8388035099626703

Classification Report: 
               precision    recall  f1-score   support

           0       0.85      0.81      0.83      9914
           1       0.82      0.86      0.8